## Extract data from random forcing experiments

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import os, sys
import copy


class data_struct:
    def __init__(self, ii, tt, name):
        self.input = ii
        self.target = tt
        self.experiment = name
        self.nstep = ii.shape[0]

def readdata_file_addp(caseid, file_ind):
    path='/n/kuangdss01/users/qiyusong/SAM_output/OUT_STAT/'
    nT = 26
    nQ = 14
    dataset = nc.Dataset(path+f'RCE_randmultsine_{caseid}{str(file_ind)}.nc', 'r')
    T = dataset['TABS'][:, :].astype(float)
    Q = dataset['QV'][:, :].astype(float)#+dataset['QCOND'][:, :].astype(float)
    P = dataset['PREC'][:].astype(float)
    P = P[:, np.newaxis]
    Y = np.concatenate((T[:, :nT], Q[:, :nQ]), axis=1)
    
    dT = dataset['TTENDR'][:, :].astype(float)
    dQ = dataset['QTENDR'][:, :].astype(float)
    U = np.concatenate((dT[:, :nT], dQ[:, :nQ]), axis=1)
    return Y, U, P

def readdata_exp_addp(casename, start_no, end_no, caseid_prefix):
    caseid = f'{caseid_prefix}_{casename}_'
    Y_list = []
    U_list = []
    LOGP_list = []
    for i in range(start_no, end_no+1):
        Y, U, P = readdata_file_addp(caseid, i);
        Y_list.append(Y)
        U_list.append(U)
        LOGP_list.append(np.log(P+1e-6))
    Y_array = np.concatenate(Y_list, axis=0)
    U_array = np.concatenate(U_list, axis=0)
    LOGP_array = np.concatenate(LOGP_list, axis=0)
    return Y_array, U_array, LOGP_array
    
def readdata_addp(experiments: list, caseid_prefix='run_256', period_mean=False, remove_mean=False):
    # prepare data
    nT = 26
    nQ = 14
    dm = np.loadtxt('IDEAL_dm.txt', dtype=np.float64)
    mass_weight = np.sqrt(np.diag(list(dm[:nT]) + list(6.25*dm[:nQ])))
    num_Exp=len(experiments)
    num_P=4        # number of periods
    len_P=19200    # length of a period (in unit of 900ss)
    if int(caseid_prefix[4:])==256:
        spinup_caseid = "spinup_256_0_"
    elif int(caseid_prefix[4:])==1024:
        spinup_caseid = "spinup_1024"
    spinup_Y, spinup_U, spinup_P = readdata_file_addp(spinup_caseid, 3)
    spinup_Y_mean = np.mean(spinup_Y, axis=0)
    spinup_LOGP_mean = np.mean(np.log(spinup_P+1e-6))
    data = []
    for i in range(len(experiments)):
        # read out the average spectra
        Y, U, LOGP = readdata_exp_addp(experiments[i], 0, 19, caseid_prefix)
        Y = Y - spinup_Y_mean
        LOGP = LOGP - spinup_LOGP_mean
        if period_mean:
            tmp = np.zeros((len_P, nT+nQ))
            for iP in range(num_P):
                tmp += Y[len_P*(iP+1):len_P*(iP+2), :]
            Y = tmp/float(num_P)
            U = U[:len_P, :]
            tmp = np.zeros((len_P, 1))
            for iP in range(num_P):
                tmp += LOGP[len_P*(iP+1):len_P*(iP+2), :]
            LOGP = tmp/float(num_P)
        if remove_mean:
            Y = Y - np.mean(Y, axis=0)
            LOGP = LOGP - np.mean(LOGP, axis=0)
        ntime = Y.shape[0]
        Y_weighted = Y * 1.
        U_weighted = U * 1.
        # turn into energy unit
        for itime in range(ntime):
            Y_weighted[itime, :] = np.matmul(Y[itime, :], mass_weight)
            U_weighted[itime, :] = np.matmul(U[itime, :], mass_weight)
        Y = np.concatenate((Y, LOGP), axis=1)
        Y_weighted = np.concatenate((Y_weighted, LOGP), axis=1)
        print(f'Saving {experiments[i]}...')
        np.savez_compressed(f'/n/kuangdss01/users/qiyusong/SAM_RNN/data/{experiments[i]}_256_addp.npz',
            input=U, target=Y,
            input_weighted=U_weighted, target_weighted=Y_weighted,
            experiment=experiments[i]
        )
        print('Saving finished.')
    #return data

In [3]:
readdata_addp(['msinefx4_0', 'msinefx4_1',
          'msinefx6_0', 'msinefx6_1',
          'msinefx8_0', 'msinefx8_1',
          'msinefx10_0', 'msinefx10_1',
          'msinefx15_0', 'msinefx15_1',
          'msinefx20_0', 'msinefx20_1',
          'msinefx30_0', 'msinefx30_1',
          'msinefx40_0', 'msinefx40_1',
          'msinefx50_0', 'msinefx50_1'
        ], caseid_prefix='run_256', period_mean=False, remove_mean=False)

Saving msinefx4_0...
Saving finished.
Saving msinefx4_1...
Saving finished.
Saving msinefx6_0...
Saving finished.
Saving msinefx6_1...
Saving finished.
Saving msinefx8_0...
Saving finished.
Saving msinefx8_1...
Saving finished.
Saving msinefx10_0...
Saving finished.
Saving msinefx10_1...
Saving finished.
Saving msinefx15_0...
Saving finished.
Saving msinefx15_1...
Saving finished.
Saving msinefx20_0...
Saving finished.
Saving msinefx20_1...
Saving finished.
Saving msinefx30_0...
Saving finished.
Saving msinefx30_1...
Saving finished.
Saving msinefx40_0...
Saving finished.
Saving msinefx40_1...
Saving finished.
Saving msinefx50_0...
Saving finished.
Saving msinefx50_1...
Saving finished.


## Extract data from coupled-wave forcing experiments

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import os, sys
import copy

def readdata_file_noisywave_addp(caseid, file_ind):
    path='/n/home03/qiyusong/SAM6.11.7/OUT_STAT/'
    nT = 26
    nQ = 14
    dataset = nc.Dataset(path+f'RCE_noisywave_{caseid}{str(file_ind)}.nc', 'r')
    T = dataset['T_WAVE'][:, :].astype(float)
    T_BG = dataset['T_WAVEBG'][:, :].astype(float)
    T = T-T_BG
    Q = dataset['Q_WAVE'][:, :].astype(float)
    Q_BG = dataset['Q_WAVEBG'][:, :].astype(float)
    Q = Q-Q_BG
    Y = np.concatenate((T[:, :nT], Q[:, :nQ]), axis=1)
    P = dataset['PREC'][:].astype(float)
    P = P[:, np.newaxis]
    
    dT = dataset['TTENDWV'][:, :].astype(float) / 96. # convert K/day to K in 15min
    dQ = dataset['QTENDWV'][:, :].astype(float) / 96. # convert g/kg/day to g/kg in 15min
    U = np.concatenate((dT[:, :nT], dQ[:, :nQ]), axis=1)
    return Y, U, P

def readdata_exp_noisywave_addp(caseid, start_no, end_no):
    Y_list = []
    U_list = []
    LOGP_list = []
    for i in range(start_no, end_no+1):
        Y, U, P = readdata_file_noisywave_addp(caseid, i);
        Y_list.append(Y)
        U_list.append(U)
        LOGP_list.append(np.log(P+1e-6))
    Y_array = np.concatenate(Y_list, axis=0)
    U_array = np.concatenate(U_list, axis=0)
    LOGP_array = np.concatenate(LOGP_list, axis=0)
    return Y_array, U_array, LOGP_array
    
def readdata_noisywave_addp(wavenumbers: list, dampings: list):
    # prepare data
    nT = 26
    nQ = 14
    dm = np.loadtxt('IDEAL_dm.txt', dtype=np.float64)
    mass_weight = np.sqrt(np.diag(list(dm[:nT]) + list(6.25*dm[:nQ])))
    
    spinup_caseid = "spinup_2_"
    spinup_Y, spinup_U, spinup_P = readdata_file_noisywave_addp(spinup_caseid, 3)
    spinup_LOGP_mean = np.mean(np.log(spinup_P+1e-6))
    
    for i_wn in range(len(wavenumbers)):
        for i_damp in range(len(dampings)):
            for i_exp in range(2):
                wn = wavenumbers[i_wn]
                damp = dampings[i_damp]
                caseid = f'wn{wn}_damp{damp}day_noadvectbg_noiselevel_1.0_{i_exp+1}_'
                # read out the average spectra
                Y, U, LOGP = readdata_exp_noisywave_addp(caseid, 1, 3)
                LOGP = LOGP - spinup_LOGP_mean
                ntime = Y.shape[0]
                Y_weighted = Y * 1.
                U_weighted = U * 1.
                # turn into energy unit
                for itime in range(ntime):
                    Y_weighted[itime, :] = np.matmul(Y[itime, :], mass_weight)
                    U_weighted[itime, :] = np.matmul(U[itime, :], mass_weight)
                Y = np.concatenate((Y, LOGP), axis=1)
                Y_weighted = np.concatenate((Y_weighted, LOGP), axis=1)
                print(f'Saving {caseid}...')
                np.savez_compressed(f'/n/kuangdss01/users/qiyusong/SAM_RNN/data/{caseid}addp.npz',
                    input=U, target=Y,
                    input_weighted=U_weighted, target_weighted=Y_weighted,
                    experiment=caseid
                )
                print('Saving finished.')

In [7]:
readdata_noisywave_addp(wavenumbers=list(range(6,21)), dampings=[2])

Saving wn6_damp2day_noadvectbg_noiselevel_1.0_1_...
Saving finished.
Saving wn6_damp2day_noadvectbg_noiselevel_1.0_2_...
Saving finished.
Saving wn7_damp2day_noadvectbg_noiselevel_1.0_1_...
Saving finished.
Saving wn7_damp2day_noadvectbg_noiselevel_1.0_2_...
Saving finished.
Saving wn8_damp2day_noadvectbg_noiselevel_1.0_1_...
Saving finished.
Saving wn8_damp2day_noadvectbg_noiselevel_1.0_2_...
Saving finished.
Saving wn9_damp2day_noadvectbg_noiselevel_1.0_1_...
Saving finished.
Saving wn9_damp2day_noadvectbg_noiselevel_1.0_2_...
Saving finished.
Saving wn10_damp2day_noadvectbg_noiselevel_1.0_1_...
Saving finished.
Saving wn10_damp2day_noadvectbg_noiselevel_1.0_2_...
Saving finished.
Saving wn11_damp2day_noadvectbg_noiselevel_1.0_1_...
Saving finished.
Saving wn11_damp2day_noadvectbg_noiselevel_1.0_2_...
Saving finished.
Saving wn12_damp2day_noadvectbg_noiselevel_1.0_1_...
Saving finished.
Saving wn12_damp2day_noadvectbg_noiselevel_1.0_2_...
Saving finished.
Saving wn13_damp2day_noadvec